In [1]:
%pip install PyPDF2 keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5

In [ ]:
##### GLOBAL UTILITY FUNCTION #####
import ast
import requests
from bs4 import BeautifulSoup

def sanitize_text(text: str) -> str:
    return text.encode("utf-8", "surrogatepass").decode("utf-8", "ignore")

def string_to_object(string:str):
    return ast.literal_eval(string.strip(";"))

def sanitize_text(text: str) -> str:
    return text.encode("utf-8", "surrogatepass").decode("utf-8", "ignore")

def string_to_object(string:str):
    return ast.literal_eval(string.strip(";"))

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

def extract_info(soup):
    # Contoh: Ekstrak teks dari semua paragraf
    txt_Arr = []
    paragraphs = soup.find_all('p')
    for para in paragraphs:
        txt_Arr.append(para.get_text())

    return txt_Arr

def process_url(url):
    html_content = fetch_html(url)
    if html_content:
        soup = parse_html(html_content)
        text = extract_info(soup)
        text = " ".join(text)
        return text
##### END-GLOBAL UTILITY FUNCTION #####

In [1]:
############## IMPORT NECESSARY LIBRARIES ###############

import os
import random
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from pydantic import BaseModel
from PIL import Image
import io
import PyPDF2
import re
from wordcloud import wordcloud
import gensim
from gensim.utils import simple_preprocess

import string
from collections import Counter
import json
import google.generativeai as genai

import time
import keras_nlp
import tensorflow as tf
import yaml

import matplotlib.pyplot as plt
from wordcloud import WordCloud
from io import BytesIO
import base64
import random

################## DEFINE THE ENV VARIABLES #####################
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["HF_TOKEN"] = 'ENTER_YOUR_HUGGINGFACE_TOKEN'
os.environ["NGROK_AUTH_TOKEN"] = "ENTER_YOUR_NGROK_TOKEN"


################## LOAD CONFIGURATIONS #####################
with open("./config.yaml", "r") as file:
    config = yaml.safe_load(file)

url = "https://huggingface.co/mnabielap/bart-multinews/resolve/main/bart-multinews.keras"
local_path = "bart-multinews.keras"
os.system(f"wget {url} -O {local_path}")

############## DEFINE GLOBAL POINTER FOR API ##############
POINTER = 0
GEMINI_API_KEY_COLLECTION = config["GEMINI_API_KEY_COLLECTION"]
random.shuffle(GEMINI_API_KEY_COLLECTION)

################### SUMMARIZER CLASS ############################
class BartSummarizer: # /summarize/bart
    def __init__(self, max_length = 256):
        self.max_length = max_length
        self.bart_model = tf.keras.models.load_model(
            local_path,
            custom_objects={"BartSeq2SeqLM": keras_nlp.models.BartSeq2SeqLM}
        )

    def summarize(self, input_text):
        output = self.bart_model.generate(input_text, max_length=self.max_length)
        return output

class GeminiLLM():
    def __init__(self, configs=config):
        self.maximum_try = 10
        self.api_key = GEMINI_API_KEY_COLLECTION
        self.generation_conf = configs["generation_config"]
        self.safety_settings = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_LOW_AND_ABOVE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_LOW_AND_ABOVE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_LOW_AND_ABOVE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_LOW_AND_ABOVE",
            },
        ]

    def pick_random_key(self):
        global POINTER
        if self.api_key:
            pair_api_key = self.api_key[POINTER]
            POINTER = (POINTER + 1) % len(self.api_key) # move to the next
            api_key, email_name = pair_api_key
            print(f"Using API Key from -> {email_name}")
            return api_key
        else:
            return "No more API keys available."

    def generate_result(self, input_text, system_instruction):
        input_text = sanitize_text(input_text)
        api_key = self.pick_random_key()
        genai.configure(api_key=api_key)

        model = genai.GenerativeModel(
            model_name="gemini-1.5-flash",
            safety_settings=self.safety_settings,
            generation_config=self.generation_conf,
            system_instruction=system_instruction,
        )

        chat_session = model.start_chat(history=[])
        response = chat_session.send_message(input_text)

        response_text = response.text
        return response_text

    ######## UTILITY FUNCTION ########
    def extract_text_from_pdf(self, pdf_path):
        pdf_reader = PyPDF2.PdfReader(pdf_path)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        return text

    def create_str_json_example(self, num_topic: int):
        res = "[\n"
        for no_topic in range(num_topic):
            no_topic += 1
            comma = ',' if no_topic != num_topic else ''
            temp = '''
            {
                "topic":"{name_topic_'''.strip() + str(no_topic) + '''}",
                "percentage":{percentage_'''.strip() + str(no_topic) + '''},
                "detail":"{explanation_'''.strip() + str(no_topic) + '''}"
            }
            '''.strip() + str(comma)
            res += f"  {temp}\n"
        res += "]"
        return res
    ##################################

class GeminiSummarizer(GeminiLLM): # /summarize/gemini
    def __init__(self, configs=config):
        super().__init__(configs)

    def process_text(self, input_text):
        system_instruction='Objective:\nYou are summarization application specifically designed for researchers to efficiently extract key information from academic papers. Researchers often face the challenge of sifting through numerous papers to find inspiration and relevant information, but only about 20% of a paper typically contains the critical insights they need. This application aims to expedite the literature review process by providing concise, targeted summaries focusing on the most valuable parts of each paper.\n\nKey Requirements:\n\nMethodology Summary: Clearly outline the research methods used, including experimental design, data collection, and analysis techniques.\n\nEquations: Highlight and extract every important equation if exists. The equations must be written in LaTeX so it can be rendered in markdown media. Do not let more than three equations on the same line, if there are more than three, put it in the new line. Make sure to always use the equation environment to write an equation that is given in a line, e.g. $$ H_ {k_p}=frac {Y_ {k_p}} {X_ {k_p}} $$. Also, make sure to be careful on writing equations from documents provided by user, because sometimes PDF breaks the latex format and you might write it wrong. Make sure to not forget every detail, for example you must write it like\n\n$$ A_{dot} = \\text{softmax} (  \n\\frac {QK^T} {\\sqrt{d_{model}}}  \n) V. $$  \n$$ A_{mem} = \\frac{σ(Q)M_{s-1}}\n{σ(Q)z_{s−1}} . $$  \n$$ M_{s} ← M_{s−1} + σ(K)^TV \\text{ and } z_{s} ← z_{s−1} + \\sum_{t=1}^{N} \nσ(K_{t}). $$  \n$$ M_{s} ← M_{s−1} + σ(K)^T(V − \\frac{σ(K)M_{s−1}}{σ(K)z_{s−1}}). $$\n\nMake sure to do deeper reasoning to implement the equation correctly, I know you render the equations from PDF directly, but use your knowledge to figure out how is it supposed to be written correctly. Do not just write what you saw directly.\n\nResults Summary: Highlight the main findings and outcomes of the research, emphasizing significant results and conclusions.\nCitations for Each Argument: Provide citations AND paper reference in APA style in the end of the summary, for key arguments and claims made within the paper to facilitate further reading and verification. The citation must be written in APA style, extract from the Reference Section in the input document if exists. REMEMBER, just provide the some needed citations and reference only, no need to provide all of the citations used on the paper.\n\nImportant Aspects of the Method: Identify and summarize critical aspects and innovations of the methodology that contribute to the research field.\n\nApplication Expectations:\n\nNon-Generic Summaries: The application should avoid general summaries (such as abstracts) and focus on specific sections that contain essential details for researchers.\nEfficiency and Accuracy: Ensure that the summarization process is fast and accurate, enabling researchers to quickly grasp the core contributions of each paper.\nUser-Centric Design: Tailor the application interface and features to meet the needs of researchers, allowing them to customize the type and depth of summaries they receive.\nOutcome:\nBy using you, researchers should be able to significantly reduce the time spent on literature reviews, thereby enhancing their productivity and enabling them to produce more research papers. The application should act as a valuable tool in accelerating the research process and improving the overall quality of academic work.\n\nYou are not allowed to answer another question aside summarization task. Expected responses:\n\nExample 1:\nUsers: "Hello"\nYou: <No response>\n\nExample 2:\nUsers: "Umm"\nYou: <No response>\n'
        response_text = self.generate_result(input_text, system_instruction)
        return response_text

    def run_gemini_summarizer(self, text, mode="text"):
        if mode == "pdf":
            text = self.extract_text_from_pdf(text)

        for _ in range(self.maximum_try):
            try:
                summary = self.process_text(text)
                return summary
            except Exception as e:
                print(f"error: {e}")
        return "Failed to summarize the text. Please try again later."

################### ANALYZER CLASS ############################
class TopicModelling(GeminiLLM):
    def __init__(self, configs=config):
        super().__init__(configs)

    def process_text(self, input_text):
        system_instruction=f"""
            Analyze the 5 most related topics in the text below. explain each topic completely. explain why the text fits the topic.
            >> Constraint:
            - minimum 100 words and maximum 500 words
            - also explain how many percent of each topic is related to the text.
            - provide in a formal, no-nonsense format so that these results can be used for academic purposes.
            - to the point
            - the output must be json format for example
            {self.create_str_json_example(5)}
            - do not provide any text outside of json
            """.strip()
        response_text = self.generate_result(input_text, system_instruction)
        return response_text

    def wordcloud(self, text):
        text = str(text).lower()
        text = re.sub(r'\d+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()
        words = [word for word in words if len(word) > 2]

        additional_stopwords = {'could', 'would', 'never', 'one', 'even', 'like', 'said', 'say', 'also',
                                'might', 'must', 'every', 'much', 'may', 'two', 'know', 'upon', 'without',
                                'go', 'went', 'got', 'put', 'see', 'seem', 'seemed', 'take', 'taken',
                                'make', 'made', 'come', 'came', 'look', 'looking', 'think', 'thinking',
                                'thought', 'use', 'used', 'find', 'found', 'give', 'given', 'tell', 'told',
                                'ask', 'asked', 'back', 'get', 'getting', 'keep', 'kept', 'let', 'lets',
                                'seems', 'leave', 'left', 'set', 'from', 'subject', 're', 'edu', 'use'}
        custom_stopwords = set(stopwords.words('english')).union(additional_stopwords)

        filtered_words = [word for word in words if word not in custom_stopwords]

        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
        word_freq = Counter(lemmatized_words)
        sorted_word_freq = sorted(word_freq.items(), key=lambda item: item[1], reverse=True)
        temp_dict = dict()
        for key, val in sorted_word_freq:
            temp_dict[key] = val
        return temp_dict

    ######################## ANDROID UTILITY ########################
    def barplot_to_base64(self, data):
        # Extracting data for plotting
        topics = [item['topic'] for item in data]
        percentages = [item['percentage'] for item in data]

        # Creating the barplot
        plt.figure(figsize=(10, 6))
        plt.barh(topics, percentages, color='skyblue')
        plt.xlabel('Percentage')
        plt.ylabel('Topic')
        plt.title('Distribution of Topics in Quantum Computing')
        plt.gca().invert_yaxis()  # Invert y-axis to have the highest percentage on top

        # Save the plot to a BytesIO object
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)

        # Encode the BytesIO object to base64
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()

        return img_base64

    def wordcloud_to_base64(self, word_freq):
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')

        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        plt.close()
        buffer.seek(0)
        img_str = base64.b64encode(buffer.read()).decode('utf-8')
        return img_str
    ########################################################################

    def run_analysis(self, text, mode="pdf", media="frontend"):
        if mode == "pdf":
            text = self.extract_text_from_pdf(text)
        elif mode == "link":
            text = process_url(text)

        for _ in range(self.maximum_try):
            try:
                topic_dist = self.process_text(text)
                break
            except Exception as e:
                print(f"error: {e}")
        try:
            topic_dist = string_to_object(topic_dist[topic_dist.index("["):topic_dist.rindex("]")+1])
        except Exception:
            topic_dist = string_to_object(topic_dist[topic_dist.index("{"):topic_dist.rindex("}")+1])

        wordcloud_dict = self.wordcloud(text)

        if mode == "android":
            topic_dist_img = self.barplot_to_base64(topic_dist)
            wordcloud_img = self.wordcloud_to_base64(wordcloud_dict)
            dict_analysis = {"topic_distribution": topic_dist_img, "wordcloud": wordcloud_img}
        else:
            dict_analysis = {"topic_distribution": topic_dist, "wordcloud": wordcloud_dict}

        return dict_analysis

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
summarizer_bart = BartSummarizer()
summarizer_gemini = GeminiSummarizer()
topic_modeller = TopicModelling()

In [3]:
text = """
In the year 2023, the small coastal town of Cape Serenity was known for its breathtaking landscapes, historic lighthouse, and vibrant community. However, what truly set Cape Serenity apart was its annual "Festival of Lights," an event that drew visitors from all over the world. This festival, held every December, transformed the town into a glowing wonderland, with every house, shop, and street decorated with millions of twinkling lights.

The origins of the Festival of Lights date back to the late 19th century. It all began when a local fisherman, Thomas Branson, decided to decorate his boat with lanterns to guide him back home through the dense fog that often enveloped the coast during winter. His initiative inspired other fishermen, and soon, the entire fleet was adorned with lights. This impromptu tradition quickly gained popularity among the townspeople, and by the turn of the century, it had evolved into a full-fledged festival.

Over the decades, the festival grew in scale and splendor. The town council took charge of organizing the event, ensuring that each year was more spectacular than the last. They introduced various attractions, including light shows, parades, and fireworks. Local businesses and residents eagerly participated, competing for the most beautifully decorated property, a coveted title that came with a significant cash prize and local fame.

The highlight of the festival was always the grand parade. Floats elaborately decorated with lights and themes from different cultures and stories made their way through the town's main street. The parade was not only a visual feast but also a showcase of Cape Serenity's diversity and unity. Each float was designed by a different community group, from the local schools and churches to the town's various cultural societies.

In 2023, the Festival of Lights was particularly special. It marked the 125th anniversary of the event, and the town council spared no effort in making it the grandest celebration yet. Months before December, preparations were already underway. Volunteers worked tirelessly, hanging lights, setting up displays, and coordinating events. The local children, excited about the upcoming festivities, practiced for their roles in the parade and various performances.

One of the most anticipated additions to the festival was the interactive light garden in the town square. This installation featured thousands of LED flowers that responded to touch and movement, creating an ever-changing sea of color and light. Visitors could walk through the garden, their steps triggering waves of illumination that followed them. It was a hit among both the young and the old, providing countless photo opportunities and moments of awe.

As December approached, the town buzzed with anticipation. The local bakery, known for its seasonal treats, unveiled a new creation: the "Lighthouse Cookie," a gingerbread replica of Cape Serenity's iconic lighthouse, complete with edible lights. The cookie quickly became a bestseller, with people lining up to get a taste.

The festival officially began on the first Friday of December, with the lighting ceremony at the town square. The mayor, accompanied by the oldest and youngest residents of Cape Serenity, flipped the switch that set the town aglow. The moment was magical, with the lights twinkling to life against the backdrop of a clear, starry night. Cheers erupted, and the festivities commenced.

Throughout the month, Cape Serenity was a hive of activity. Tourists flooded in, filling up the local inns and restaurants. The town's economy received a significant boost, and the sense of community pride was palpable. There were concerts, street performances, and workshops where visitors could learn to make their own lanterns and decorations.

The grand parade, held on the third Saturday of December, was the crowning jewel of the festival. This year's theme was "Journey Through Time," celebrating the history of the festival and the town. Floats depicted scenes from different eras, from the humble beginnings with Thomas Branson's lantern-lit boat to futuristic visions of Cape Serenity. The creativity and effort put into each float were evident, and the parade drew record crowds.

As the festival drew to a close on New Year's Eve, there was a sense of accomplishment and joy in the air. The final event was a spectacular fireworks display over the harbor, illuminating the night sky in a brilliant array of colors. Families gathered to watch, reflecting on the month of festivities and the sense of togetherness it had fostered.

The 125th Festival of Lights in Cape Serenity was a resounding success, a testament to the town's enduring spirit and the power of tradition to bring people together. As the lights dimmed and the new year dawned, the residents of Cape Serenity looked forward to the future, proud of their heritage and excited for the many festivals yet to come.
"""

In [11]:
result = summarizer_gemini.run_gemini_summarizer(
    text="/content/AnimeGANv3_manuscript.pdf",
    mode="pdf"
)
print(result)

Using API Key from -> shopee-gmail-2
## Summary of "A Novel Double-Tail Generative Adversarial Network for Fast Photo Animation"

This research proposes a novel generative adversarial network (GAN) called DTGAN (Double-Tail GAN) for fast photo animation, meaning converting real-world photos into anime-style images. 

**Methodology Summary:**

* **DTGAN Structure:** The generator in DTGAN has two output tails - a support tail for generating coarse-grained anime images and a main tail for refining them. 
* **Linearly Adaptive Denormalization (LADE):** A novel learnable normalization technique is introduced to prevent artifacts in generated images. LADE uses a point-wise convolution to obtain global data distribution information of features, guiding instance normalization and preventing artifacts.
* **Loss Functions:**
    * **Region Smoothing Loss:** Uses superpixel segmentation to weaken complex texture details in the generated images, achieving an abstract anime effect.
    * **Fine-Gr

In [6]:
result = topic_modeller.run_analysis(
    text="/content/AnimeGANv3_manuscript.pdf",
    mode="pdf", media="frontend"
)
print(json.dumps(result, indent=4))

Using API Key from -> muhammad.nabiel11@ui.ac.id
